# Import The necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# loading, preparing and cleaning the data

In [ ]:
#step 1: Loading the dataset
df = pd.read_csv('Statewide Solar Projects.csv', low_memory=False)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

1. The average estimated PV system size is 24.5 kWdc, while the actual PV system size is 19.1 kWac, indicating efficiency losses between the two. 

2. The mean estimated annual energy production is around 28,768 kWh, though there is high variability, with a standard deviation of 342,204 kWh. 

3. Most projects are small, as indicated by the 50th percentile values for PV system size and energy production, but there are a few larger installations with maximum values of 43,470 kWdc and 51,026,820 kWh. 

4. The majority of the projects do not have energy storage systems.

In [ ]:
columns_to_drop = ['Data Through Date', 'Project ID', 'Circuit ID', 'Developer', 'Energy Storage System Size (kWac)', 'Number of Projects']
df = df.drop(columns=columns_to_drop)

Dropping these columns because they are not essential for our project, and kWac has a not missing values which as mentioned suggests that most of the projects dont have energy storage systems

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# Step 2: Impute missing values for 'City/Town' using the most frequent value within each 'County'
# Create a function to fill missing City/Town values based on the most frequent City/Town in that County
def fill_city_town(row, df):
    if pd.isnull(row['City/Town']):
        return df[df['County'] == row['County']]['City/Town'].mode()[0]
    else:
        return row['City/Town']

In [ ]:
df['City/Town'] = df.apply(lambda row: fill_city_town(row, df), axis=1)

In [ ]:
# Step 3: Impute missing values for 'Zip' using the most frequent Zip code in each 'City/Town'
def fill_zip(row, df):
    if pd.isnull(row['Zip']):
        return df[df['City/Town'] == row['City/Town']]['Zip'].mode()[0]
    else:
        return row['Zip']

In [ ]:
df['Zip'] = df.apply(lambda row: fill_zip(row, df), axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
# Step 4: Impute missing values for 'Division' using the most frequent value within the same 'Zip'
def fill_division(row, df):
    if pd.isnull(row['Division']):
        mode_value = df[df['Zip'] == row['Zip']]['Division'].mode()
        if len(mode_value) > 0:
            return mode_value[0]  # Use the most frequent Division
        else:
            return np.nan  # Return NaN if no mode is found
    else:
        return row['Division']

In [ ]:
df['Division'] = df.apply(lambda row: fill_division(row, df), axis=1)

In [ ]:
# Step 5: Impute missing values for 'Substation' using the most frequent value within the same 'Zip'
def fill_substation(row, df):
    if pd.isnull(row['Substation']):
        mode_value = df[df['Zip'] == row['Zip']]['Substation'].mode()
        if len(mode_value) > 0:
            return mode_value[0]  # Use the most frequent Substation
        else:
            return np.nan  # Return NaN if no mode is found
    else:
        return row['Substation']

In [ ]:
df['Substation'] = df.apply(lambda row: fill_substation(row, df), axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
# Step 6: Impute missing values for 'Metering Method' using the most frequent value (mode)
df['Metering Method'] = df['Metering Method'].fillna(df['Metering Method'].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
# Drop the 'Division' column since it has too many missing values
df = df.drop(columns=['Division'])

In [ ]:
# Drop rows where 'Substation' is null
df = df[df['Substation'].notna()]

In [ ]:
# Check the updated number of missing values
print(df.isnull().sum())

In [ ]:
df.shape

In [ ]:
df_encoded = df.copy()

making a copy of the dataframe because we need encoding and we cannot retrieve the information while geoplotting

In [ ]:
# Step 7: Encoding categorical variables
city_encoder = LabelEncoder()
county_encoder = LabelEncoder()
substation_encoder = LabelEncoder()
zip_encoder = LabelEncoder()

In [ ]:
df_encoded['City/Town_encoded'] = city_encoder.fit_transform(df_encoded['City/Town'])
df_encoded['County_encoded'] = county_encoder.fit_transform(df_encoded['County'])
df_encoded['Substation_encoded'] = substation_encoder.fit_transform(df_encoded['Substation'])
df_encoded['Zip_encoded'] = zip_encoder.fit_transform(df_encoded['Zip'].astype(str))

In [ ]:
# Use One-Hot Encoding for lower-cardinality categorical features
df_encoded = pd.get_dummies(df_encoded, columns=['Utility', 'Metering Method'], drop_first=True)

In [ ]:
# Step 8: Convert 'Interconnection Date' to numeric (years since connection)
df_encoded['Interconnection Date'] = pd.to_datetime(df_encoded['Interconnection Date'])
df_encoded['Interconnection Age'] = (pd.to_datetime('today') - df_encoded['Interconnection Date']).dt.days / 365.25
df_encoded = df_encoded.drop(columns=['Interconnection Date'])  # Drop the original date column

In [ ]:
df_encoded = df_encoded.drop(columns=['Zip', 'City/Town', 'County', 'Substation'])

In [ ]:
df_encoded.head()

In [ ]:
df_encoded.shape

In [ ]:
# Step 9: Scale numerical features
scaler = MinMaxScaler()

In [ ]:
# Select numerical columns for scaling
numerical_columns = ['Estimated PV System Size (kWdc)', 'PV System Size (kWac)', 'Estimated Annual PV Energy Production (kWh)', 'Interconnection Age']
df_encoded[numerical_columns] = scaler.fit_transform(df_encoded[numerical_columns])

# Clustering Algorithms

### 1. K-Means Clustering

In [ ]:
def evaluate_metrics(df_encoded, min_clust=2, max_clust=8, rand_state=42):
    inertias = []
    silhouette = []
    for n_clust in range(min_clust, max_clust):
        kmeans = KMeans(n_clusters=n_clust, random_state=rand_state)
        y_label = kmeans.fit_predict(df_encoded)
        inertias.append(kmeans.inertia_)
        silhouette.append(silhouette_score(df_encoded, y_label))   
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))

    ax[0].plot(range(min_clust, max_clust), inertias, '-x', linewidth=2)
    ax[0].set_xlabel('No. of clusters')
    ax[0].set_ylabel('Inertia')

    ax[1].plot(range(min_clust, max_clust), silhouette, '-x', linewidth=2)
    ax[1].set_xlabel('No. of clusters')
    ax[1].set_ylabel('Silhouette Score')

In [ ]:
evaluate_metrics(df_encoded, min_clust=2, max_clust=8, rand_state=42)

In [ ]:
X = df_encoded.values

for n in range(2, 8):
    kmeans = KMeans(n_clusters=n, random_state=42)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(f"For n_clusters = {n}, the silhouette score is {silhouette_avg:.4f}")

In [ ]:
silhouette_scores = [] 

for n_cluster in range(2, 8):
    silhouette_scores.append( 
        silhouette_score(df_encoded, KMeans(n_clusters = n_cluster).fit_predict(df_encoded))) 
    
k = [2, 3, 4, 5, 6,7] 
plt.bar(k, silhouette_scores) 
plt.xlabel('Number of clusters', fontsize = 10) 
plt.ylabel('Silhouette Score', fontsize = 10) 
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit_predict(df_encoded)
labels = kmeans.labels_

In [ ]:
cluster_centers = pd.DataFrame(data = kmeans.cluster_centers_, columns = [df_encoded.columns])
cluster_centers

In [ ]:
cluster_centers_scaled = cluster_centers[numerical_columns]
cluster_centers_inverse = scaler.inverse_transform(cluster_centers_scaled)
cluster_centers[numerical_columns] = cluster_centers_inverse
cluster_centers

In [ ]:
df_cluster_with_kmean = pd.concat([df_encoded, pd.DataFrame({'kmeans_cluster': labels})], axis = 1)
df_cluster_with_kmean

In [ ]:
df_cluster_with_kmean.shape

In [ ]:
df_cluster_with_kmean.isnull().sum()

In [ ]:
ax = sns.countplot(x=df_cluster_with_kmean.kmeans_cluster)
ax.bar_label(ax.containers[0], fontsize=12, color='black', fontweight='bold')
plt.show()

In [ ]:
df_kmean = df_encoded.copy()

pca = PCA(n_components = 2)
principal_comp = pca.fit_transform(df_kmean)
pca_df = pd.DataFrame(data = principal_comp, columns = ['pca1', 'pca2'])
pca_df.head()

In [ ]:
pca_df = pd.DataFrame(data = principal_comp, columns = ['pca1', 'pca2'])

pca_df = pd.concat([pca_df, pd.DataFrame({'kmeans_cluster':labels})], axis=1)
pca_df.head()

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x='pca1', y='pca2',hue = 'kmeans_cluster', data = pca_df, palette=['red','green','blue','yellow','orange'], alpha=0.75)
plt.title('Projection of the dataset on the 2 PCA dimensions')
plt.show()

### 2. DBSCAN

In [ ]:
neigh = NearestNeighbors(n_neighbors=4)
nbrs = neigh.fit(df_encoded)
distances, indices = nbrs.kneighbors(df_encoded)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
eps_values = np.arange(0.5, 1.2, 0.2)
min_samples_values = range(2, 5)
best_score = -1
best_params = {'eps': None, 'min_samples': None}

In [ ]:
from joblib import Parallel, delayed

def evaluate_dbscan(eps, min_samples, df_encoded):
    try:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(df_encoded)
        if len(set(labels)) > 1:  # Proceed only if more than one cluster
            score = silhouette_score(df_encoded, labels)
            return score, eps, min_samples
    except Exception as e:
        print(f"Error with eps={eps} and min_samples={min_samples}: {e}")
    return -1, eps, min_samples  # Return -1 if there's an error or only 1 cluster

# Use Parallel processing to search for the best parameters
results = Parallel(n_jobs=10, timeout=3000)(delayed(evaluate_dbscan)(eps, min_samples, df_encoded)
                                          for eps in eps_values
                                          for min_samples in min_samples_values)

In [ ]:
for score, eps, min_samples in results:
    if score > best_score:
        best_score = score
        best_params['eps'] = eps
        best_params['min_samples'] = min_samples

# Output the best parameters and silhouette score
print(f"Best silhouette score: {best_score}")
print(f"Best parameters: eps={best_params['eps']}, min_samples={best_params['min_samples']}")


In [ ]:
print(f"Best Silhouette Score: {best_score}")
print(f"Optimal Epsilon: {best_params['eps']}")
print(f"Optimal Min Samples: {best_params['min_samples']}")

In [ ]:
dbscan = DBSCAN(eps=0.9, min_samples=2)
dbscan.fit(df_encoded)

In [ ]:
dbscan_labels = dbscan.labels_

In [ ]:
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

In [ ]:
print(f"Estimated number of clusters: {n_clusters}")
print(f"Estimated number of noise points: {n_noise}")

In [ ]:
pca = PCA(n_components=2)
df_reduced = pca.fit_transform(df_encoded)

In [ ]:
unique_labels = set(dbscan_labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

In [ ]:
plt.figure(figsize=(10, 7))
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise
        col = [0, 0, 0, 1]

    class_member_mask = (dbscan_labels == k)

    xy = df_reduced[class_member_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title(f'Estimated number of clusters: {n_clusters}')
plt.show()

### 3. Gaussian Mixture Model

In [ ]:
score_gmm = []
for k in range(2,8):
    gmm = GaussianMixture(n_components=k, random_state=1 )
    gmm.fit(df_encoded)
    score = silhouette_score(df_encoded, gmm.predict(df_encoded), metric='euclidean',random_state=42)
    score_gmm.append(score)
plt.plot(range(2,8), score_gmm, marker = '^', c = 'b', ms = 9, mfc = 'r')
plt.xticks(range(2,8))
plt.xlabel("Number of Clusters")
plt.ylabel("silhouette coefficients")
plt.show() 

In [ ]:
covariance_types = ['full', 'tied', 'diag', 'spherical']
best_silhouette = -1
best_covariance_type = None

for cov_type in covariance_types:
    gmm = GaussianMixture(n_components=3, covariance_type=cov_type, random_state=42)
    gmm_labels = gmm.fit_predict(X)
    silhouette_avg = silhouette_score(X, gmm_labels)
    print(f"For covariance_type = {cov_type}, the silhouette score is {silhouette_avg:.4f}")
    
    if silhouette_avg > best_silhouette:
        best_silhouette = silhouette_avg
        best_covariance_type = cov_type

print(f"Best silhouette score: {best_silhouette:.4f} with covariance_type = {best_covariance_type}")

In [ ]:
gmm = GaussianMixture(n_components=3, covariance_type='spherical', random_state=42)
gmm_labels = gmm.fit_predict(df_encoded)

In [ ]:
cluster_means = pd.DataFrame(data = gmm.means_, columns = [df_encoded.columns])
cluster_means

In [ ]:
cluster_means_scaled = cluster_means[numerical_columns]
cluster_means_inverse = scaler.inverse_transform(cluster_means_scaled)
cluster_means[numerical_columns] = cluster_means_inverse
cluster_means

In [ ]:
df_cluster_with_gmm = pd.concat([df_encoded, pd.DataFrame({'gmm_cluster': gmm_labels})], axis = 1)
df_cluster_with_gmm

# Model Comparison

In [ ]:
kmeans_eval = KMeans(n_clusters=5, random_state=42).fit_predict(df_encoded)
gmm_eval = GaussianMixture(n_components=3, covariance_type='spherical', random_state=40).fit_predict(df_encoded)

In [ ]:
kmeans_silhouette = silhouette_score(df_encoded, kmeans_eval)
gmm_silhouette = silhouette_score(df_encoded, gmm_eval)

results_df = pd.DataFrame({
    'Model': ['KMeans', 'Gaussian Mixture'],
    'Number of Clusters': [5, 3],
    'Silhouette Score': [kmeans_silhouette, gmm_silhouette]
})


In [ ]:
results_df

In [ ]:
results_melted = pd.melt(results_df, id_vars=['Model'], value_vars=['Number of Clusters', 'Silhouette Score'])
plt.figure(figsize=(14, 8))
sns.barplot(x='Model', y='value', hue='variable', data=results_melted, palette='viridis')
plt.title('Clustering Model Evaluation Metrics')
plt.xlabel('Clustering Model')
plt.ylabel('Metric Value')
plt.show()

# Visualization

In [ ]:
df = pd.concat([df, pd.DataFrame({'kmeans_cluster': labels})], axis = 1)

In [ ]:
df = df.drop(columns=['Interconnection Date'])

In [ ]:
df = pd.concat([df, df_encoded['Interconnection Age']], axis=1)
df

In [ ]:
df.info()

In [ ]:
df=df.dropna()

In [ ]:
df.head()

In [ ]:
new_labels = kmeans.labels_

cluster_summary = df.groupby('kmeans_cluster').agg({
    'Estimated PV System Size (kWdc)': ['mean', 'median', 'max', 'min'],
    'PV System Size (kWac)': ['mean', 'median', 'max', 'min'],
    'Estimated Annual PV Energy Production (kWh)': ['mean', 'median', 'max', 'min'],
    'Interconnection Age': ['mean', 'median', 'max', 'min'],
    'City/Town': 'count',  # Count the number of projects in each city/town
    'County': 'count',
    'Zip': 'count'
})

# Print cluster summaries
cluster_summary

In [ ]:
zip_lat_lon = pd.read_csv('uszips.csv')

In [ ]:
zip_lat_lon.head()

In [ ]:
df = pd.merge(df, zip_lat_lon[['zip', 'lat', 'lng']], left_on='Zip', right_on='zip', how='left')

In [ ]:
df

In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

In [ ]:
cluster_colors = {
    0: 'blue',
    1: 'green',
    2: 'red',
    3: 'purple',
    4: 'orange'
}

In [ ]:
df_clean = df.dropna(subset=['lat', 'lng'])

In [ ]:
# Initialize the folium map centered around New York State
m = folium.Map(location=[40.7128, -74.0060], zoom_start=10)  # New York City coordinates

In [ ]:
# Create a marker cluster for each cluster
for cluster in df_clean['kmeans_cluster'].unique():
    # Create a marker cluster group for each kmeans cluster
    marker_cluster = MarkerCluster(name=f'Cluster {int(cluster)}').add_to(m)
    
    # Add points to the cluster group based on the cluster
    cluster_data = df_clean[df_clean['kmeans_cluster'] == cluster]
    for i, row in cluster_data.iterrows():
        # Use the color map to assign a color based on the cluster number
        marker_color = cluster_colors.get(row['kmeans_cluster'], 'blue')  # Default to blue if not found

        folium.Marker(
            location=[row['lat'], row['lng']],
            popup=f"{row['City/Town']}, {row['County']}\nPV Size: {row['PV System Size (kWac)']} kWac\nEnergy Production: {row['Estimated Annual PV Energy Production (kWh)']} kWh",
            icon=folium.Icon(color=marker_color)
        ).add_to(marker_cluster)


In [ ]:
# Add layer control to toggle clusters
folium.LayerControl().add_to(m)

In [ ]:
# Save the map as an HTML file
m.save("solar_plants_clusters.html")

In [ ]:
# To display the map within a notebook (if needed)
m

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Load the full US states GeoJSON
gdf = gpd.read_file('us-states.json')

# Filter for New York state
ny_state_gdf = gdf[gdf['name'] == 'New York']

# Save as GeoJSON
ny_state_gdf.to_file('ny_state.geojson', driver='GeoJSON')

# Read the newly saved New York State GeoJSON file
ny_state_geojson = 'ny_state.geojson'
ny_state_gdf = gpd.read_file(ny_state_geojson)

In [ ]:
# Generate random points within the state's boundary
minx, miny, maxx, maxy = ny_state_gdf.total_bounds  # Get bounding box of NY State

# Number of random points to generate
num_points = 5000
points = []

# Generate points until we have enough within the state boundary
while len(points) < num_points:
    # Generate random latitude and longitude within the bounding box
    lon = np.random.uniform(minx, maxx)
    lat = np.random.uniform(miny, maxy)
    point = Point(lon, lat)

    # Check if the point is within the state boundary
    if ny_state_gdf.contains(point).values[0]:
        points.append([lon, lat])

# Convert the points to a NumPy array
points = np.array(points)

# Apply KMeans clustering to the points
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(points)
labels = kmeans.labels_

# Add the clusters back into a GeoDataFrame
clustered_points_gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lon, lat in points])
clustered_points_gdf['cluster'] = labels

# Plot the clusters (Optional: for visualization)
plt.scatter(points[:, 0], points[:, 1], c=labels, cmap='tab10')
plt.show()

# Create a Folium map centered on New York State
m = folium.Map(location=[42.9, -75.5], zoom_start=7)  # Rough center of New York State

# Define the cluster colors
cluster_colors = {
    0: 'blue',
    1: 'green',
    2: 'red',
    3: 'purple',
    4: 'orange'
}

# Plot each cluster's points on the Folium map
for i in range(5):
    cluster_data = clustered_points_gdf[clustered_points_gdf['cluster'] == i]
    for point in cluster_data['geometry']:
        folium.CircleMarker(
            location=[point.y, point.x],  # Folium uses (lat, lon)
            radius=2,
            color=cluster_colors[i],
            fill=True,
            fill_color=cluster_colors[i],
            fill_opacity=0.6
        ).add_to(m)

# Optionally add the New York state boundary to the map
folium.GeoJson(ny_state_geojson, name="New York State Boundary").add_to(m)

# Add layer control to toggle clusters
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save("ny_state_clusters.html")

# To display the map in a notebook (if applicable)
m

In [ ]:
plt.figure(figsize=(10, 6))
df.groupby('kmeans_cluster')['Estimated Annual PV Energy Production (kWh)'].mean().plot(kind='bar')
plt.title('Average Annual PV Energy Production by Cluster')
plt.ylabel('Average Energy Production (kWh)')
plt.show()

In [ ]:
palette = sns.color_palette("hls", 5)

In [ ]:
# First plot for 'Estimated PV System Size (kWdc)'
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, 
                x='Estimated PV System Size (kWdc)', 
                y='kmeans_cluster', 
                hue='kmeans_cluster', 
                palette=palette, 
                markers=["o", "s", "D", "v", "^"])
plt.title('Estimated PV System Size (kWdc) vs Clusters')
plt.show()

In [ ]:
# Second plot for 'PV System Size (kWac)'
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, 
                x='PV System Size (kWac)', 
                y='kmeans_cluster', 
                hue='kmeans_cluster', 
                palette=palette, 
                markers=["o", "s", "D", "v", "^"])
plt.title('PV System Size (kWac) vs Clusters')
plt.show()

In [ ]:
# Third plot for 'Estimated Annual PV Energy Production (kWh)'
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, 
                x='Estimated Annual PV Energy Production (kWh)', 
                y='kmeans_cluster', 
                hue='kmeans_cluster', 
                palette=palette, 
                markers=["o", "s", "D", "v", "^"])
plt.title('Estimated Annual PV Energy Production (kWh) vs Clusters')
plt.show()

**Cluster 0 (Red):**

*Profile:*
Cluster 0 contains solar projects with generally smaller values across most features compared to other clusters. These projects have lower PV system sizes (both kWdc and kWac) and relatively low energy production levels.

*Behavior:*

`PV System Size (kWdc and kWac):` Projects in this cluster typically have small systems, ranging up to around 4,000 kW for both PV System Size (kWac) and 
Estimated PV System Size (kWdc). Most of the projects are on the smaller side, with a significant concentration below 2,000 kW.

`Estimated Annual PV Energy Production (kWh):` This cluster’s energy production is similarly low, with most systems producing less than 2 million kWh annually, indicating these projects are likely small residential or low-capacity commercial solar installations.

*Possible Characteristics:*
These projects are likely smaller, perhaps residential or small-scale commercial solar installations.
Given the lower PV system sizes and energy production, these projects might be in regions with lower energy demand or funding for large-scale installations.

**Cluster 1 (Green):**

*Profile:*
Cluster 1 features moderate-sized projects in terms of PV system size and energy production. These projects have higher system capacities and produce more energy than those in Cluster 0, but they still remain mid-range compared to other clusters.

*Behavior:*

`PV System Size (kWdc and kWac):` The solar systems in this cluster generally range from 1,000 kW to 4,000 kW, indicating mid-sized projects.

`Estimated Annual PV Energy Production (kWh):` Energy production values for this cluster fall between approximately 2 million and 4 million kWh. This suggests these projects are larger than residential systems but still smaller than the largest utility-scale projects.

*Possible Characteristics:*
These are likely small-to-medium commercial installations or community solar projects that produce enough energy to serve small towns or industrial areas.
This group might include organizations or small businesses investing in solar energy to reduce electricity costs and achieve moderate energy independence.

**Cluster 2 (Blue):**

*Profile:*
Cluster 2 stands out as having larger PV systems and higher energy production values than Clusters 0 and 1. This cluster includes more substantial projects with system sizes between moderate and large-scale installations.

*Behavior:*

`PV System Size (kWdc and kWac):` Systems in Cluster 2 have capacities between 3,000 and 5,000 kW, indicating significant solar installations that can supply substantial energy.

`Estimated Annual PV Energy Production (kWh):` Energy production for this cluster ranges widely, with most systems producing around 4 to 6 million kWh annually. These projects appear to be high-output solar installations.

*Possible Characteristics:*
These are likely larger commercial or small utility-scale projects designed to serve industrial areas or larger communities.
This cluster might represent projects that are critical to regional energy production but aren't the largest installations in the state.

**Cluster 3 (Purple):**

*Profile:*
Cluster 3 consists of projects with the highest system sizes and energy production levels. These projects represent the largest solar installations in the dataset, with significant PV capacities and energy outputs.

*Behavior:*

`PV System Size (kWdc and kWac):` This cluster contains systems that reach up to 10,000 kW (or 10 MW), reflecting large solar farms or utility-scale projects.

`Estimated Annual PV Energy Production (kWh):` These projects have the highest energy production values, with output often exceeding 10 million kWh annually. Some systems approach 15 million kWh in production, marking them as the largest producers in the state.

*Possible Characteristics:*
These are utility-scale solar farms, designed to generate vast amounts of electricity for the grid, serving large urban centers or powering significant portions of the state's energy demand.
Projects in this cluster are likely to be highly funded and may represent government or corporate-backed initiatives to generate large-scale renewable energy.

**Cluster 4 (Orange):**

*Profile:*
Cluster 4 includes mid-to-high capacity projects, falling between Cluster 2 and Cluster 3 in terms of system size and energy production. These projects are larger than average but not as large as the very largest installations.

*Behavior:*

`PV System Size (kWdc and kWac):` Projects in this cluster are generally between 3,000 and 8,000 kW in size, suggesting substantial but not the largest projects.

`Estimated Annual PV Energy Production (kWh):` Production levels for this cluster range between 4 to 8 million kWh, indicating strong energy output, though not at the level of utility-scale farms.

*Possible Characteristics:*
This cluster may represent large commercial or industrial solar projects, as well as some utility-scale solar installations.
These projects likely play an important role in local energy supply but do not reach the scale or output of the largest solar farms found in Cluster 3.
